# Double Pendulum

## Table of Contents

- [Batches and the Data Structure](#batches-and-the-data-structure)
- [Query Data](#query-data)
- [Initial Parameters](#initial-parameters)
- [Pendulum Motion Animation](#pendulum-motion-animation)
- [Plot Trajectories. Variations in Initial Angles](#plot-trajectories-variations-in-initial-angles)
- [Statistics](#statistics)
- [Standard Deviation Boundary Test](#standard-deviation-boundary-test)

## Batches and the Data Structure

We conducted simulations of the double pendulum system with varying initial parameters:

- In the batch 'double_pendulum_angles' we varied the initial angles of the first pendulum by selecting values randomly from a normal distribution with a mean of 120 degrees and a standard deviation of 5 degrees.

- In the batch 'double_pendulum_vel' we varied the initial velocity of the second pendulum, choosing values randomly from a normal distribution with a mean of 90 degrees per second and a standard deviation of 5 degrees per second.

In [ ]:
from citros_data_analysis import data_access as da

Let's create CitrosDB object to query and plot the results of the simulations. Let's specify that we would like to see batches that were created with double pendulum simulation scenario:

In [ ]:
citros = da.CitrosDB(simulation = 'simulation_double_pendulum')

To print information about all batches created in 'simulation_double_pendulum' simulation, call `search_batch()`:

In [ ]:
citros.search_batch().print()

Let's print the names of the batches containing simulations with the status 'DONE'. This indicates that the simulations have successfully finished:

In [ ]:
list(citros.search_batch(sid_status='DONE').keys())

Print general information about the most recent simulation. In this simulation we have two topics: '/config' and '/coordinates':

In [ ]:
citros.batch(-1).info().print()

Topic '/config' contains initial parameters of the simulation, topic '/coordinates' contains result of the simulation. Let's look on data structure of the topic '/coordinates':

In [ ]:
citros.batch(-1).topic('/coordinates').info().print()

As we stated in the README, the result of the simulation has the following structure: there is time coordinate 'data.t' and two coordinates of the two pendulums: data.p1.x, data.p1.y, data.p2.x, data.p2.y; there is also a record about the type of the ros message:

In [ ]:
citros.batch(-1).topic('/coordinates').info()['topics']['/coordinates']['data_structure']['data'].print()

## Query Data

Let's query data by `data()` method. If we call `data()` method without arguments we get all data separated by columns.


The output of the `data()` method is a pandas.DataFrame, so every method of the pandas.DataFrame can be applied to the result of the query.
Here by `head()` method we left only first 5 rows of the output:

In [ ]:
citros.batch(-1).topic('/coordinates').data().head(5)

We can query not all data, but, for example, only time and coordinates of the second pendulum:

In [ ]:
citros.batch(-1).topic('/coordinates').data(['data.t', 'data.p2.x', 'data.p2.y']).head(5)

## Initial Parameters

In '/config' topic we can find all the initial parameters. It is convenient to get them as a dictionary:

In [ ]:
name = 'data.double_pendulum.ros__parameters'
params = citros.batch(-1).topic('/config').data(name, additional_columns='sid')
params = params[params[name].notna()].set_index('sid')[name]

print(f"pandas.Series with initial parameters for the simulation runs:\n{params}")
print(f"\ndict with paramaters for the sid = 0:\n{params[0]}")
print(f"\ninitial angle for the sid = 0:\na1_0 = {params[0]['a1_0']}")

## Pendulum Motion Animation

Let's query data and plot the animation of the pendulum motion for the first 5 second of one of the simulation run, for example for sid = 0:

In [ ]:
F = citros.batch(-1).sid(0).set_filter({'data.t': {'<=': 5}}).topic('/coordinates').data()

In [ ]:
import matplotlib.pyplot as plt
from collections import deque

def animate(F):
    # length of the history trace
    trace_len = 100

    max_y_0 = abs(min(F['data.p2.y']))*1.2
    max_y_1 = max([max(F['data.p1.y']), max(F['data.p2.y'])])*1.2
    max_x = max([max(abs(F['data.p1.x'])), max(abs(F['data.p2.x'])), 0.1])*1.2
    fig = plt.figure()
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-max_x, max_x), ylim=(-max_y_0, max_y_1))
    ax.set_aspect('equal', 'datalim')
    ax.grid()

    line1, = ax.plot([], [], 'o-', lw=2)
    line2, = ax.plot([], [], 'o-', lw=2)
    trace, = ax.plot([], [], 'b-', lw=0.5, ms=1)
    time_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
    trace_x, trace_y = deque(maxlen=trace_len), deque(maxlen=trace_len)

    plt.close()
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        trace.set_data([], [])
        time_text.set_text('')
        trace_x.clear()
        trace_y.clear()
        return line1, line2, trace, time_text

    def animate_frame(i):
        trace_x.appendleft(F['data.p2.x'].iloc[i])
        trace_y.appendleft(F['data.p2.y'].iloc[i])

        line1.set_data([[F['data.p1.x'].iloc[i],F['data.p2.x'].iloc[i]],[F['data.p1.y'].iloc[i], F['data.p2.y'].iloc[i]]])
        line2.set_data([[0,F['data.p1.x'].iloc[i]],[0, F['data.p1.y'].iloc[i]]])
        trace.set_data(trace_x, trace_y)

        time_text.set_text('Time = %.3f s' % F['data.t'].iloc[i])
        return line1, line2, trace, time_text
    
    init()
    for i in range(len(F)):
        animate_frame(i)
        display(fig, clear=True)

animate(F)

The animation playback speed depends on the length of the data. If the animation is too slow, you just can decrees the number of messages in query by averaging or picking every n-th message. For example, to select every 5th message:

In [ ]:
F = citros.batch(-1).sid(0).skip(5).set_filter({'data.t': {'<=': 5}}).topic('/coordinates').data()
animate(F)

## Plot Trajectories. Variations in Initial Angles

Small variations in the initial parameters of the double pendulum system lead to significant changes in trajectories.
Let's plot the trajectories for simulations where either initial angle or initial velocities are varied.

Previously we made a simulation with different initial angles of the first pendulum.
We can print the initial angles using the '/config' topic:

In [ ]:
col_name = 'data.double_pendulum.ros__parameters.a1_0'

a1_0 = citros.batch(-1).topic('/config').data(col_name, additional_columns='sid').rename({col_name: 'a1_0'}, axis = 1)
a1_0 = a1_0 [a1_0 ['a1_0'].notna()].set_index('sid')
a1_0

Let's plot the trajectory of the second pendulum from the different simulations:

In [ ]:
F = citros.batch(-1).topic('/coordinates').data()
citros.plot_graph(F, 'data.p2.x', 'data.p2.y')

Let's plot the trajectory of the second pendulum from the first and second simulations:

In [ ]:
F = citros.batch(-1).topic('/coordinates').sid([1,2]).data()
citros.plot_graph(F, 'data.p2.x', 'data.p2.y')

## Statistics

We can examine how the coordinates of the second pendulum, along with their mean values and standard deviations, change over time using the `citros_data_analysis.error_analysis` package:

In [ ]:
from citros_data_analysis import error_analysis as analysis

F = citros.batch(-1).topic('/coordinates').data()
dataset = analysis.CitrosData(F, data_label=['data.p2.x', 'data.p2.y'], units = 'm')
db = dataset.bin_data(n_bins = 50, param_label = 'data.t')
db.show_statistics()

Let's plot the scatter plot of the last points of the simulation (around 'data.t' = 10). Let's also depict 1-, 2-, 3-sigma error ellipses and print parameters of the 1-sigma ellipse:

In [ ]:
ellipse_param = db.show_correlation(x_col = 'data.p2.x', y_col = 'data.p2.y',
                      slice_val = 10, n_std = [1,2,3], return_ellipse_param = True)
ellipse_param[0]

## Standard Deviation Boundary Test

By utilizing different tests from the `citros_data_analysis.validation` package, you can quickly determine whether the simulation meets specific conditions. For instance, let's verify if the standard deviation remains within certain limits. Suppose we want to check if the mean value of the coordinates for the second pendulum, along with its 1-sigma standard deviation boundary, falls within a 2.5 x 2.5 box for the 'double_pendulum_angle' batch:

In [ ]:
from citros_data_analysis import validation as va

F = citros.batch(-1).topic('/coordinates').data()
V = va.Validation(F, data_label = ['data.p2.x', 'data.p2.y'], param_label = 'data.t', method = 'bin', num = 50, units = 'm')
log, table, fig = V.std_bound_test(limits = 2.5, n_std = 1, std_area = True)

In [ ]:
ref = da.Ref()
ref.print()